In [37]:
import requests
from bs4 import BeautifulSoup
import warnings
import pandas as pd
warnings.filterwarnings('ignore', message='A NumPy version.*"')

# Set \# and Name Extraction

In [38]:
# base variables, set to site
base_url = 'https://jp.pokellector.com/'
url = 'https://jp.pokellector.com/Pokemon-151-Expansion/'
html = requests.get(url)
soup = BeautifulSoup(html.text, 'html.parser')

In [39]:
# get all card numbers and names
soup_elements = soup.find_all(class_='plaque')

In [40]:
# append info into a list
pokemon_list = []
for soup_element in soup_elements:
    curr_list = soup_element.text.split('-')
    pokemon_list.append([curr_list[0][:-1], curr_list[1][1:]])

# Rarity Extraction

In [30]:
rarity_url = 'https://www.tcgcollector.com/cards/jp/pokemon-card-151?releaseDateOrder=newToOld&cardsPerPage=30&displayAs=images'
rarity_html = requests.get(rarity_url)
rarity_soup = BeautifulSoup(rarity_html.text, 'html.parser')
rarity_soup_elements = rarity_soup.find_all('img', class_='card-rarity-symbol')

In [31]:
rarity_list = []
for rarity_soup_element in rarity_soup_elements:
    curr_rarity = rarity_soup_element['alt']
    filtered_rarity = curr_rarity.split()[-1].strip('()')
    rarity_list.append(filtered_rarity)

C
U
RR
C
U
RR
C
U
RR
C
C
U
C
C
R
C
C
U
C
U
C
U
C
RR
C
R
C
U
C
U
U
C
U
R
C
U
C
RR
C
RR
C
U
C
U
R
C
U
C
U
C
U
C
U
C
U
C
U
C
U
C
C
U
C
U
RR
C
U
R
C
C
U
C
U
C
U
RR
C
U
C
U
C
U
C
C
R
C
U
C
U
C
U
C
U
R
U
C
U
C
U
C
R
C
U
C
R
U
U
C
C
R
C
U
R
C
RR
C
U
C
U
C
R
R
U
RR
C
C
U
U
C
R
U
R
C
R
R
R
C
U
R
U
R
R
U
R
RR
R
C
U
R
R
RR
U
U
C
C
C
U
U
U
U
U
U
U
U
U
AR
AR
AR
AR
AR
AR
AR
AR
AR
AR
AR
AR
AR
AR
AR
AR
AR
AR
SR
SR
SR
SR
SR
SR
SR
SR
SR
SR
SR
SR
SR
SR
SR
SR
SAR
SAR
SAR
SAR
SAR
SAR
SAR
SAR
UR
UR
UR


In [45]:
# append rarity_list elements to pokemon list
for i in range(len(pokemon_list)):
    pokemon_list[i].append(rarity_list[i])

[['#1', 'Bulbasaur', 'C'], ['#2', 'Ivysaur', 'U'], ['#3', 'Venusaur ex', 'RR'], ['#4', 'Charmander', 'C'], ['#5', 'Charmeleon', 'U'], ['#6', 'Charizard ex', 'RR'], ['#7', 'Squirtle', 'C'], ['#8', 'Wartortle', 'U'], ['#9', 'Blastoise ex', 'RR'], ['#10', 'Caterpie', 'C'], ['#11', 'Metapod', 'C'], ['#12', 'Butterfree', 'U'], ['#13', 'Weedle', 'C'], ['#14', 'Kakuna', 'C'], ['#15', 'Beedrill', 'R'], ['#16', 'Pidgey', 'C'], ['#17', 'Pidgeotto', 'C'], ['#18', 'Pidgeot', 'U'], ['#19', 'Rattata', 'C'], ['#20', 'Raticate', 'U'], ['#21', 'Spearow', 'C'], ['#22', 'Fearow', 'U'], ['#23', 'Ekans', 'C'], ['#24', 'Arbok ex', 'RR'], ['#25', 'Pikachu', 'C'], ['#26', 'Raichu', 'R'], ['#27', 'Sandshrew', 'C'], ['#28', 'Sandslash', 'U'], ['#29', 'Nidoran ♀', 'C'], ['#30', 'Nidorina', 'U'], ['#31', 'Nidoqueen', 'U'], ['#32', 'Nidoran ♂', 'C'], ['#33', 'Nidorino', 'U'], ['#34', 'Nidoking', 'R'], ['#35', 'Clefairy', 'C'], ['#36', 'Clefable', 'U'], ['#37', 'Vulpix', 'C'], ['#38', 'Ninetales ex', 'RR'], ['#39',

# Output into .xlsx file

In [47]:
# put data into data frame and output
df = pd.DataFrame(pokemon_list, columns=['Set #', 'Name', 'Rarity'])
df.to_excel('sv2a.xlsx',index=False)